# Subject Data PreProcessing
* find subject data
* concatenate the inter-session data
* concatenate the intra-session data

# TO DO
* Adapt the cleaning cell to also consider empty data points and mark them as noData in the DataFrame
* Write gaze definition cell by counting identical collider appearances while 21 consecutive hits are then defined as a gaze caused by an actual fixation. 


## Configuration

In [1]:
# General configuration
import os

# data_directory: str
#     Path to a directory to store data.
data_directory = '.'

# install_missing_packages: bool
#     A flag indicating if missing packages should be automatically installed
install_missing_packages = True

# use_conda: bool
#     A flag indicating if conda should be used for software installation.
#     If False, pip will be used. The default is to use conda if jupyter
#     is run in a conda environment.
use_conda = 'CONDA_EXE' in os.environ

## Checking for missing packages

In [2]:
import importlib

def check_package(package, pip_pkg: str = None, conda_pkg: str = None):
    """Check if a given package is installed. If missing install
    it (if global flag `install_missing_packages` is True) either with
    pip or with conda (depending on `use_conda`).
    """
    if importlib.util.find_spec(package) is not None:
        return  # ok, package is already installed

    if not install_missing_packages:
        raise RuntimeError(f"{package} is not installed!")

    if use_conda:
        import conda.cli
        conda.cli.main('conda', 'install',  '-y', conda_pkg or package)
    else:
        import subprocess
        import sys            
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_pkg or package])
        
# This is to exit cells without error tracebacks (cosmetic purpose)
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Creating the required environment (skip if already done)

Running the following cell will create a file graphs.yml that can be used to setup a conda environment containing the required packages. If you already downloaded the file from my GitHub, skip the next cell and create the env directly from it.

In [3]:
%%writefile graphs.yml
name: graphs
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.6
  - jupyter
  - imageio
  - imageio-ffmpeg
  - matplotlib
  - scikit-image
  - opencv
  - networkx
  - pandas
  - statsmodels

Writing graphs.yml


### Environment Creation
To create the environment, open the terminal, go to the directory where you stored the graphs.yml file (the directory of the notebook) and type
conda env create -f graphs.yml
After running this command you have to activate the environment (Linux/MacOS: conda activate graphs, Windows: activate graphs) and then reopen the notebook in that environment.

## Main Part

### Imports and directory information

In [8]:
import os
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nX
import glob
import scipy.cluster.vq as clusters
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import normalize
from pandas.plotting import autocorrelation_plot as AC_plot 
#from statsmodels.graphics import tsaplots
#from statsmodels.tsa.stattools import acf
from skimage.filters import gaussian
from mpl_toolkits.mplot3d import Axes3D 
import time




In [21]:
OG_DATA_PATH = './'
condition_single = False #True for Single, False for dyadic
filename_keyword = 'Single' if condition_single else 'Dyadic'
DATA_PATH = './Data {}/'.format(filename_keyword)

PROCESSED_DATA_PATH = './Results/'

### Extracting all subject IDs from the data folder

In [29]:
# Getting the Folder without hidden files in ascending order 
DATA_FOLDER = sorted([f for f in os.listdir(DATA_PATH) if not f.startswith('.')], key=str.lower)

subIDs = []
for sub in DATA_FOLDER:
    if sub[0].isdigit():
        subIDs.append(int(sub[0:4]))
    else:
        pass
subIDs = np.unique(subIDs)
print(subIDs)

[1023]


# Combining Pathfinding Experiment Files per Subject

In [30]:
EXPT_FILE_ATTRS = ['ParticipantID', 'PathsReversed', 'IsDyadic', 'IsLeader'];
#subIDs = [1023] # remove to do for all subIDs


def map_dict(a_dictionary):
    a_subset = {key: value for key, value in a_dictionary.items() if key in EYE_FILE_ATTRS};
    return a_subset;

# For each subject in subject ID folder, combine all experiment files
# and save as a unified final experiment file in Result folder
for sub in subIDs:
    # Get all files for subject
    SUB_FILES = list(filter(lambda x: str(sub) in x, DATA_FOLDER));
    
    # Sort files into experiment files and eyetracking files
    EYETRACKING_FILES = list(filter(lambda x: 'EyeTracking' in x, SUB_FILES));
    EYETRACKING_FILES.sort();
    
    EXPT_FILES = list(filter(lambda x: filename_keyword + 'PathFinding' in x, SUB_FILES));
    EXPT_FILES.sort();
    
    EXPT_DATA = {};
        
    # For each experiment file
    for expt_file in EXPT_FILES:
        
        # Read JSON data
        with open(DATA_PATH + expt_file) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + expt_file + " is not valid!")
                continue;
        
        # If there are more than one experiment files, combine the data of the paths if the 
        # other trial and subject information matches
        if len(EXPT_DATA) > 0:
            if(not all([subject_session[x] == EXPT_DATA[x] for x in EXPT_FILE_ATTRS])):
                raise Exception("Experiment File Data " + expt_file + " Does Not Match!")
            else:
                EXPT_DATA['Concatenated'] = True;
                EXPT_DATA['TrialData'] += subject_session['TrialData'];
        
        else:
            EXPT_DATA = subject_session;
            EXPT_DATA['Concatenated'] = False;
        
    # Check if correct number of paths present in experiment file
    if len(EXPT_DATA['TrialData']) != 10:
        raise Exception("Incorrent number of paths in file for Subject " + str(sub));
        break;
        
    # Save the final combined file in results to be accessed by eyetracking collection script
    expt_file_name = PROCESSED_DATA_PATH + str(sub) + "_"+filename_keyword+"Pathfinding_Final.json";
    with open(expt_file_name, 'w') as fp:
        json.dump(EXPT_DATA, fp);
    

In [25]:
# Leader : follower map
dyadic_pair_map = {
    '1021' : '1022',
    '1023' : '1013',
    '1005' : '1055',
    '1074' : '1069',
    '1008' : '1058',
    '1054' : '1004',
    '1011' : '1017',
    '1018' : '1057'
}

### Combine the data
* Loop through all subjects
* extract the session data
* combine the data
* save it

In [35]:
# --------- Preparation ---------

Session_save_bool = False # set to True if you want to save each individual session as csv
Exploration_save_bool = True # set to True if you want to save the complete exploration as csv
subcount = 0

# column name list for dataframe
col_names =  ['PathNumber',
          'timeStampDataPointStart',
          'timeStampDataPointEnd',
          'serverTimeStampDataPointStart',
          'serverTimeStampDataPointEnd',
          'hitObjectColliderName', 
          'ordinalOfHit',
          'BitMask',
          'hitPointOnObject.x',
          'hitPointOnObject.y',
          'hitPointOnObject.z',
          'hitObjectColliderBoundsCenter.x',
          'hitObjectColliderBoundsCenter.y',
          'hitObjectColliderBoundsCenter.z']


# TODO: Add server times to nohitdict
NoHit_dict = {'hitPointOnObject': {'x': 'NaN',
                                   'y': 'NaN',
                                   'z': 'NaN'},
              'hitObjectColliderName': 'NoHit',
              'hitObjectColliderBoundsCenter': {'x': 'NaN',
                                                'y': 'NaN',
                                                'z': 'NaN'},
              'ordinalOfHit': 'NaN',
              'Session': 'NaN',
              'timeStampDataPointStart': 'NaN',
              'timeStampDataPointEnd': 'NaN',
              'serverTimeStampDataPointStart' : 'NaN',
              'serverTimeStampDataPointEnd' : 'NaN',
              'BitMask': 'NaN'
              }

# Remove machine time stamps if single
if condition_single:
    col_names.pop(3);
    col_names.pop(3);
    NoHit_dict.pop('serverTimeStampDataPointStart');
    NoHit_dict.pop('serverTimeStampDataPointEnd');



# --------------------------- MAIN PART ---------------------------


# --------- first layer - subject loop ---------

for subject in subIDs:
    subcount +=1
    print('Subject ' + str(subject) + ' started - ' + str(subcount) + '/' + str(len(subIDs)))
    
    # Create empty dataframe for later concatenation
    complete_exploration_df = pd.DataFrame(columns = col_names)
    
    # get the data files according to the subject
    subject_folder = sorted([f for f in DATA_FOLDER 
                             if f.startswith(str(subject)+'_EyeTrackingData_')], 
                            key=str.lower) 
    
    # the following works as long as the data name format is as follows:
    # 'subjectID'_Expl_S_'SessionNumber'_ET_'EyeTrackingSessionNumber'_'UnixTimestamp'.json
    folder_files = list()
       
    # loop through the subject folder and save all numbers
    for file in subject_folder:
        folder_files.append(re.findall(r'\d+', file))
    
    if len(folder_files) == 0:
        raise Exception("No eyetracking files in " + filename_keyword +" condition for subject " + str(sub))
    # Extract all SubIDs (only one), (and Timestamps)
    SubID, UnixTimestamp1, _ = map(list, zip(*folder_files))
    
    # Get the experiment JSON file for the subject
    sub_expt_data = {}
    sub_expt_file_name = PROCESSED_DATA_PATH + str(sub) + "_"+filename_keyword+"Pathfinding_Final.json";
    try:
        with open(sub_expt_file_name, 'r') as fp:
            sub_expt_data = json.load(fp);
    except:
        raise Exception("Could not read experiment file for subject " + str(sub));
    
# --------- second layer - exploration session loop ---------
    s = 0;
    complete_hitpoints_df = pd.DataFrame(columns = col_names)
    previous_path_num = -1.0;
    # loop over separate eye tracking sessions
    for ET_session in subject_folder:
        s+=1
        #print("\tSession " + str(s) + " started")

            # open the JSON file as dictionary
        with open(DATA_PATH + ET_session) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + ET_session + " is not valid!")

        hitpoint_list = list() # create hitpoint list
        
        # check if trial is valid:
        trial_valid = subject_session['trials'][0]['trialIsValid']
        if not trial_valid:
            raise Exception("Excluding " + ET_Session + " because trial is not valid!");
            continue;
        
        sub_id = subject_session['trials'][0]['participantId']
        if str(sub_id) != str(subject):
            raise Exception("Excluding " + ET_Session + " because subject IDs don't match!");
            continue;
        
        # start timestamp of the session 
        start_time = subject_session['trials'][0]['timeTrialMeasurementStarted']
        
        # get trial id (path number)
        path_num = subject_session['trials'][0]['trialId']
        
        expt_path_start_time = sub_expt_data['TrialData'][path_num]['StartMachineTimeStamp'];
        expt_path_end_time = sub_expt_data['TrialData'][path_num]['EndMachineTimeStamp'];
        
        machine_time_stamp_diff = 0.0;
        if not condition_single and path_num != previous_path_num:
            expt_path_start_srvtime_mS = sub_expt_data['TrialData'][path_num]['StartServerTimeStamp'];
            machine_time_stamp_diff = expt_path_start_time - expt_path_start_srvtime_mS / 1000;
            previous_path_num = path_num;

        # amount of datapoints 
        Len_subses = len(subject_session['trials'][0]['dataPoints'])

        # for loop appending each data point rayCastHit Data
        # afterwards adding the timestamp to the dict 
        # passing if there is (1) no raycast hit and (2) if there is only one raycast hit
        for each in subject_session['trials'][0]['dataPoints']:

            # account for noHits 
            if each['rayCastHitsCombinedEyes'] == []:
                hitpoint_list.append(NoHit_dict)
            else:
                # append data point
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][0]) 
                # add path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = path_num
                
                # Exclude the data point if it lies outside of the times for a path
                if each['timeStampDataPointStart'] < expt_path_start_time or each['timeStampDataPointStart'] > expt_path_end_time:
                    hitpoint_list[idx]['PathNumber'] += 100;
#                     print(str(idx) + " Data Pt. Start: " + str(each['timeStampDataPointStart']));
#                     print(str(idx) + " Data Pt. End: " + str(each['timeStampDataPointEnd']));
#                     print("Path Start: " + str(expt_path_start_time));
#                     print("Path End: " + str(expt_path_end_time));
#                     print('\n')

                # Add the server time stamp if dyadic
                if not condition_single:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']


            try: 
                # append data point of second raycast hit if it exists
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][1])
                # add Path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = hitpoint_list[idx-1]['PathNumber'];
                
                # Add the server time stamp if dyadic
                if not condition_single:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']

            except:
                pass

        # normalize the hitpoint dictionary to get dataframe
        hitpoints_df = pd.json_normalize(hitpoint_list)

        print("\tET: " + str(s) + " normalized")

        complete_hitpoints_df = complete_hitpoints_df.append(hitpoints_df)


    # --------- Saving each Session ---------

    # If you want to save each session separately, set 'Session_save_bool' to True
    if Session_save_bool == True:
        try:
            if len(subject_data) > 0:
                complete_hitpoints_df.to_csv(PROCESSED_DATA_PATH
                                             + str(subject)
                                             + "_CompleteSession"
                                             + "_" + filename_keyword
                                             + "_Hitpoints.csv")
                print("\t"
                      + str(subject)
                      + " session "
                      + "_" + filename_keyword
                      + " saved ")
            else: 
                print("\t"
                      + str(subject)
                      + " - Session "
                      + "_" + filename_keyword
                      + " is empty!")
        except:
            print("\tCould not save subject "
                  + str(subject)
                  + " session "
                  + "_" + filename_keyword
                  + "!")



    # fill the complete exploration dataframe with the separate session data (combining the sessions)
    complete_exploration_df = complete_exploration_df.append(complete_hitpoints_df)
        
    # --------- Saving the Exploration ---------
    
    # If you want to save the exploration file, set 'Exploration_save_bool' to True
    if Exploration_save_bool == True:
        # saving the complete exploration
        try:
            complete_exploration_df.to_csv(PROCESSED_DATA_PATH + str(subject) + "_Complete"+filename_keyword+"_Hitpoints.csv")
            print("\t" + str(subject) + " " + filename_keyword + " eyetracking data saved")
        except:
            print("\tCould not save subject " + str(subject) + " " + filename_keyword + " eyetracking data!")
    

print('Done')

Subject 1023 started - 1/1
	ET: 1 normalized
	ET: 2 normalized
	ET: 3 normalized
	ET: 4 normalized
	ET: 5 normalized
	ET: 6 normalized
	ET: 7 normalized
	ET: 8 normalized
	ET: 9 normalized
	ET: 10 normalized
	1023 Dyadic eyetracking data saved
Done


In [36]:
Result_df = pd.read_csv(PROCESSED_DATA_PATH+'1023_CompleteDyadic_Hitpoints.csv')

In [37]:
Result_df

,Unnamed: 0,PathNumber,timeStampDataPointStart,timeStampDataPointEnd,serverTimeStampDataPointStart,serverTimeStampDataPointEnd,hitObjectColliderName,ordinalOfHit,BitMask,hitPointOnObject.x,hitPointOnObject.y,hitPointOnObject.z,hitObjectColliderBoundsCenter.x,hitObjectColliderBoundsCenter.y,hitObjectColliderBoundsCenter.z,Session
0,0,100.0,0.001985,0.008432,1.488637e+09,1.488637e+09,Female_063_NoHands,1.0,3.0,-286.073700,1.188752,-161.713196,-285.866089,1.320987,-161.400894,NaN
1,1,100.0,0.001985,0.008432,1.488637e+09,1.488637e+09,pavement_C.002,2.0,3.0,-282.921753,0.137312,-157.582443,-217.756897,-2.905127,-288.595245,NaN
2,2,100.0,0.008432,0.008432,1.488637e+09,1.488637e+09,Female_063_NoHands,1.0,3.0,-286.073700,1.188752,-161.713196,-285.866089,1.320987,-161.400894,NaN
3,3,100.0,0.008432,0.008432,1.488637e+09,1.488637e+09,pavement_C.002,2.0,3.0,-282.921753,0.137312,-157.582443,-217.756897,-2.905127,-288.595245,NaN
4,4,0.0,0.013393,0.016368,1.488637e+09,1.488637e+09,Female_063_NoHands,1.0,3.0,-286.080048,1.191041,-161.749207,-285.873413,1.319996,-161.436264,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157472,19347,109.0,95.967685,95.970661,1.489627e+09,1.489627e+09,pavement_G.005,1.0,3.0,266.072479,-2.608216,180.609634,273.366241,-2.745627,186.282379,NaN
157473,19348,109.0,95.970661,95.970661,1.489627e+09,1.489627e+09,pavement_G.005,1.0,3.0,266.072479,-2.608216,180.609634,273.366241,-2.745627,186.282379,NaN
157474,19349,109.0,95.978597,95.981574,1.489627e+09,1.489627e+09,pavement_G.005,1.0,3.0,266.030792,-2.609457,180.711060,273.366241,-2.745627,186.282379,NaN
157475,19350,109.0,95.990006,95.992981,1.489627e+09,1.489627e+09,pavement_G.005,1.0,3.0,265.978119,-2.609941,180.776611,273.366241,-2.745627,186.282379,NaN


In [38]:
print("data points excluded for 1023: ");
(Result_df['PathNumber'] > 20).value_counts()


data points excluded for 1023: 


False    156839
True        638
Name: PathNumber, dtype: int64